Web Scrapping & Database connection

In [ ]:
#Enumerate function Code
#enumerate give us index for the data with the values
students=['Farhan','Abhishek','Abhinav','Tejashvi']
for index_values, data in enumerate(students):
    if 1<=index_values<=2:
        print(data)

In [ ]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

all_data=[] #used later

url="https://en.wikipedia.org/wiki/List_of_Indians_by_net_worth"
response=r.get(url)
html_data=response.content #returns html data from the url
soup=BeautifulSoup(html_data)
print('Done')


#soup.find_all('table',class='wikitable sortable jquery-tablesorter') #library couldn't find the content of this class
hello=soup.find_all('tr') #We want to find table row (tr) tag


#enumerate give us index for the data with the values
for index_no, value in enumerate(soup.find_all('tr')): 
    if 0<=index_no <=24: #Need rows from 0 to 24
        # print(index_no,value.text)
        # print(value.text.splitlines()) #This will give us empty data
        temp_data=value.text.splitlines()
        while '' in temp_data: #This will remove empty data/ blank spaces
            temp_data.remove('')
        # print(temp_data)
        all_data.append(temp_data)


print('Done')

#Now make a DataFrame of this list all_data
cols=all_data[0]
final_data=all_data[1:]
final_cols=['Rank','Name','Net Worth','Company',"Source of Wealth",'None','None']

listing=[]
hello=len(final_data)
for i in range(1,hello+1):
    listing.append(i)


# rich_list_df=pd.DataFrame(final_data, index=listing ,columns=final_cols)
# rich_list_df.index.name="Rank"
# final_df=rich_list_df.drop(["Rank","None"], axis=1) #This will display the data too
# display(final_df)

rich_list_df=pd.DataFrame(final_data ,columns=final_cols)
final_df=rich_list_df.drop(["None"], axis=1) #This will display the data too
display(final_df)

final_df.to_excel('Richest_People_in_India.xlsx') #export to excel file
print('Done')

Now Build a SQL connection to send excel to database

In [ ]:
import pymysql as py
conn=py.connect(user='root',
    password='1234',
    host='localhost',
    autocommit=True) #autosave
print('Connection created successfully')


cur=conn.cursor()


In [ ]:
query='''create database wiki;'''
cur.execute(query)
print('Done')


query='''use wiki'''
cur.execute(query)



Make columns of the table suitable for database


In [ ]:
#we want all the column names in a list to make a column for a table
table_column=['Rich_'+i.replace(' ','_')  for i in final_df.columns] #replace space with _ to put this in database
#table_column is a list
query_part=' varchar(225), '.join(table_column) + ' varchar(225)'

Create Table rich_list

In [ ]:
query='''Create table rich_list ('''+query_part+');'
print(query)
cur.execute(query)

Insert Data into Table

In [ ]:
for i in final_df.itertuples():
    values=(tuple(i)[1:])
    query=f'''insert into rich_list values {values}''' 
    # print(query)
    cur.execute(query)
print('Done')




To Close the Connection

In [ ]:
conn.close()

To Delete the Database

In [ ]:
import pymysql as py

# Establish the connection (you can reuse your existing connection object)
# Make sure to connect WITHOUT specifying the database you want to drop.
# Connect to the MySQL server itself.
conn = py.connect(user='root',
                  password='1234', # Replace with your password if different
                  host='localhost',
                  autocommit=True)

# Create a cursor
cur = conn.cursor()

# --- Code to Drop the Database ---
# The 'IF EXISTS' part prevents an error if the database doesn't exist.
query = '''DROP DATABASE IF EXISTS wiki;'''

try:
    cur.execute(query)
    print("Database 'wiki' dropped successfully (if it existed).")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # It's good practice to close the connection when you're done
    conn.close()
